# K-MCI
### Intro
In this notebook we have the main algorithm of the article. All necessary functions are implemented in function.py file.<br>
This document have blow sections:<br>
1. Importing Libraries & classes
2. Reading datasets
3. Preprocessing datasets
4. Main Algorithm

### Importing Libraries & classes
We import some libraries from third party.

In [70]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import random
import copy

In [71]:
# defnition of Candidate class
class Candidate:    
    def __init__(self,nodes,sampling_interval, variation_count,fitness=-1):
        self.nodes = nodes
        self.sampling_interval = sampling_interval 
        self.variation_count = variation_count
        self.random_number
        self.fitness = fitness
        self.centers = []
        self.probability = []
        self.labels = []
        self.each_fitness = []
        
    def random_number(self):
        self.random_number = random.random()
        
    def describe(self):
        print("Candidate with #{} nodes, variation_count={}, #{} centers, fitness={}\n clusters:\n{} \n Centers:\n{}\n sampling_interval:\n{}"
              .format(len(self.nodes),self.variation_count,len(self.centers),self.fitness,
                      self.nodes[np.random.randint(self.nodes.shape[0], size=5), :],self.centers,self.sampling_interval))

### Reading Datasets and Preprocessing
We show the structure of our data for all of datasets.<br>
We have these datasets from UCI Machine Learning Repository:
1. Breast Cancer Wisconsin
2. Contraceptive Method Choice data
3. Glass
4. Iris
5. Vowel
6. Wine

In [72]:
""" In this cell we have some code which will do preprocessing for us on all datasets """
""" the resourses of these datasets are mentioned in README.md file. """

path = 'datasets/' 
# because we are using local files, you need to download datasets and change the "path" variable
# local folder of your downloaded datasets


def bcw(): 
    # importing dataset
    dataset = pd.read_csv(path+'bcw.csv',
                          names = ['Sample code number','Clump Thickness','Uniformity of Cell Size',
                                   'Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial',
                                   'Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class'])
    dataset= dataset.replace(to_replace='?',value=np.nan)
    dataset = dataset.dropna(axis =0) # resolving missing values
    dataset = dataset.astype('int64') # casting string valued column to int64
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def cmc():
    # importing dataset
    dataset = pd.read_csv(path+'cmc.csv', names=["Wife's age","Wife's education","Husband's education",
                                                 "Number of children ever born","Wife's religion",
                                                 "Wife's now working?","Husband's occupation",
                                                 "Standard-of-living index","Media exposure","Contraceptive method used"])
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def glass():
    # importing dataset
    dataset = pd.read_csv(path+'glass.csv', names= ['Id','refractive index','Sodium','Magnesium','Aluminum','Silicon','Potassium'
                                                    ,'Calcium','Barium','Iron','glass'])
    x = dataset.iloc[:,1:10].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def iris():
    # importing dataset
    dataset = pd.read_csv(path+'iris.csv', names= ['sepal length','sepal width','petal length','petal width','class'])
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    
    # encoding categorial data types to labelEncoder
    from sklearn.preprocessing import LabelEncoder
    labelencoder_y = LabelEncoder()
    labelencoder_y = labelencoder_y.fit(y)
    y = labelencoder_y.transform(y)  # 0 for 'Iris-setosa', 1 for 'Iris-versicolor', 2 for 'Iris-virginica'
    return (x,y,dataset)


def vowel():
    # importing dataset
    dataset = pd.read_csv(path+'vowel.csv', names= ['vowel','type 1 frq','type 2 frq','type 3 frq'])
    x = dataset.iloc[:,1:].values # features
    y = dataset.iloc[:,0].values # target values
    return (x,y,dataset)

def wine():
    # importing dataset
    dataset = pd.read_csv(path+'wine.csv', names= ['class','Alcohol','Malic acid','Ash','Alcalinity of ash','Magnesium','Total phenols','Flavanoids','Nonflavanoid phenols','Proanthocyanins','Color intensity','Hue','OD280/OD315','Proline'])
    x = dataset.iloc[:,1:].values # features
    y = dataset.iloc[:,0].values # target values
    return (x,y,dataset)

In [73]:
# Getting data from our preprocessing class Datasets
x_bcw, y_bcw, df_bcw = bcw()
x_cmc, y_cmc, df_cmc = cmc()
x_glass, y_glass, df_glass = glass()
x_iris, y_iris, df_iris = iris()
x_vowel, y_vowel, df_vowel = vowel()
x_wine, y_wine, df_wine = wine()

In [74]:
# Showing dataset of Breast Cancer Wisconsin
df_bcw.head(3)

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2


In [75]:
# Showing dataset of Contraceptive Method Choice data
df_cmc.head(3)

,Wife's age,Wife's education,Husband's education,Number of children ever born,Wife's religion,Wife's now working?,Husband's occupation,Standard-of-living index,Media exposure,Contraceptive method used
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1


In [76]:
# Showing dataset of Glass
df_glass.head(3)

,Id,refractive index,Sodium,Magnesium,Aluminum,Silicon,Potassium,Calcium,Barium,Iron,glass
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1


In [77]:
# Showing dataset of Iris
df_iris.head(3)

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


In [78]:
# Showing dataset of Vowel
df_vowel.head(3)

,vowel,type 1 frq,type 2 frq,type 3 frq
0,1,700,1500,2600
1,1,550,1550,2400
2,1,700,1500,2600


In [79]:
# Showing dataset of Wine
df_wine.head(3)

,class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185


# Main Algorithm 

The implementation of main algorithm have below sections:
1. <Strong>Initializing Parameters and Candidates</Strong>
    1. Sampling Interval
    2. Apply Random Centers to Candidates
    3. Visualizing Candidates
2. <Strong>Running Kmean to Exploit</Strong>
    1. K-means Algorithm
    2. Visualizing Candidates
3. <Strong>Mutation Algorithm</Strong>
    1. Fitness over Mutated Candidate
    2. Probablility Equation
    3. Mutation Logic

**Extra details will be explain in other next sections**


## Initializing Parameters and Candidates
In this step, we initialize our algorithm. There are some parameters which are important to convergance speed and quality of solution. So we initialize them here based on the paper.<br>
Each time you want to run K-MCI algorithm, you should first do this initialization.<br>

In [80]:
# notice: this values related to IRIS dataset
candidates_array = [] # the array of all candidates
number_of_candidates = 5
sampling_interval_reduction_factor = 0.95
convergence_parameter = None # what is this in paper???
mutation_random = 0.7
iterations_count = 3500 # is this NFE in the article??? see table 2
variations_count = 15
number_of_clusters = 3 # in this article number of clusters are predefined

## Sampling Interval
this function use for making sampling_interval


In [81]:
def sampling_intervals(input_array):
    Maxes = np.amax(input_array,axis=0)
    Mins  = np.amin(input_array,axis=0)
    return (Maxes,Mins)

In [82]:
# initializing candidates
for i in range(0 ,number_of_candidates):
    candidates_array.append(Candidate(x_iris,sampling_interval = sampling_intervals(x_iris),variation_count = variations_count))

## Visualizing Candidates
some parameters are uninitialized. They will have value after running Kmean and Cohort and mutations.

In [83]:
candidates_array[random.randint(0,number_of_candidates-1)].describe()

Candidate with #150 nodes, variation_count=15, #0 centers, fitness=-1
 clusters:
[[7.7 2.6 6.9 2.3]
 [5.  3.5 1.3 0.3]
 [6.5 3.  5.8 2.2]
 [5.5 4.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]] 
 Centers:
[]
 sampling_interval:
(array([7.9, 4.4, 6.9, 2.5]), array([4.3, 2. , 1. , 0.1]))


## Apply Random Centers to Candidates
In this step, we just select some random nodes as centers with psuedorandom functions in python native library.

In [84]:
for c in range(0,number_of_candidates):
    candidates_array[c].centers =  np.asarray(random.sample(list(candidates_array[c].nodes),number_of_clusters))
    print(candidates_array[c].centers)

[[5.8 2.8 5.1 2.4]
 [5.6 2.5 3.9 1.1]
 [4.6 3.4 1.4 0.3]]
[[6.2 2.8 4.8 1.8]
 [5.9 3.2 4.8 1.8]
 [5.1 3.7 1.5 0.4]]
[[5.5 2.5 4.  1.3]
 [6.1 3.  4.9 1.8]
 [6.7 3.3 5.7 2.5]]
[[4.6 3.6 1.  0.2]
 [6.4 3.2 4.5 1.5]
 [7.3 2.9 6.3 1.8]]
[[5.2 3.4 1.4 0.2]
 [6.1 2.8 4.7 1.2]
 [5.1 3.8 1.9 0.4]]


## K-means Algorithm 
this function fitting k-means model on dataset

In [85]:
def doKmeans(x,clusters_count,init_centers):
    kmeans = KMeans(n_clusters = clusters_count, init = init_centers,n_init = 50)
    y_predict_kmeans = kmeans.fit_predict(x)
    return (y_predict_kmeans,kmeans.cluster_centers_,kmeans.inertia_)


## Running Kmean to Exploit
Here by running Kmean on all of our candidates, they centers initialized and nodes assigns to clusters based on Kmean++ algorithm.<br>
The objective function of Kmean is also the object function of K-MCI. <br>
So the fitness of any candidates is calculated using Kmean inertia or mean square error of nodes to their centers distances.

In [86]:

for c in range(0,number_of_candidates):
    y_predict_temp,centers_temp,fitness_temp = doKmeans(candidates_array[c].nodes,clusters_count = number_of_clusters,init_centers = candidates_array[c].centers)
    candidates_array[c].centers = centers_temp
    candidates_array[c].fitness = fitness_temp
    candidates_array[c].labels   = y_predict_temp
    
candidates_array[2].labels

C:\Users\HaMeD\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=50
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1])

## Visualizing Candidates
In this step, by Kmeans, we assigned centers to each candidates and calculated the fitness of each candidate using inertia

In [87]:
candidates_array[random.randint(0,number_of_candidates-1)].describe()

Candidate with #150 nodes, variation_count=15, #3 centers, fitness=78.8556658259773
 clusters:
[[4.8 3.4 1.9 0.2]
 [5.  3.5 1.3 0.3]
 [6.  2.2 4.  1. ]
 [6.4 3.2 4.5 1.5]
 [5.5 2.4 3.7 1. ]] 
 Centers:
[[6.85384615 3.07692308 5.71538462 2.05384615]
 [5.88360656 2.74098361 4.38852459 1.43442623]
 [5.006      3.428      1.462      0.246     ]]
 sampling_interval:
(array([7.9, 4.4, 6.9, 2.5]), array([4.3, 2. , 1. , 0.1]))


## Fitness over Mutated Candidate
Just simple sum of square of distances

In [88]:
def sum_of_squares(data, centroids, labels):
    sqe = 0
    for l in np.unique(labels):
        data_l = data[labels == l]
        resid = data_l - centroids[l]
        sqe += (resid**2).sum()
    return sqe

### Mean Square Distance to a Specific Center
This function is exactly based on "sum_of_squares" function. But here all centers not engaged because we want to find the sum of square distances of nodes of a cluster to their cluster center.<br>
The idea is when mutation process done, at least one dimension may get better center but the other goes bad. So we want to have that center which is better to the previous time.

In [89]:
def center_dist(data, center, labels,center_position):
    sqe = 0
    for l in np.unique(labels):
        data_l = data[labels == l][center_position]
        resid = data_l - center
        sqe += (resid**2).sum()
    return sqe

In [90]:
def each_fitness(candidates_array_example):
    fitness_of_each_centers = [[0 for x in range(len(candidates_array_example.centers[0]))]for y in range(len(candidates_array_example.centers))]
    for i in range(0,len(candidates_array_example.centers)):
        for j in range(0,len(candidates_array_example.centers[i])):
            fitness_of_each_centers[i][j] = center_dist(candidates_array_example.nodes,candidates_array_example.centers[i][j],candidates_array_example.labels,j)
    return np.array(fitness_of_each_centers)
print(each_fitness(candidates_array[0]))
print(candidates_array[0].centers)
print(len(candidates_array[0].centers))
print(len(candidates_array[0].centers[0]))


[[183.85863905  45.32946746 110.37976331  67.07786982]
 [117.25224133  48.07917227  58.83535071  92.93291051]
 [ 76.465232    45.350208    94.520528   168.320592  ]]
[[6.85384615 3.07692308 5.71538462 2.05384615]
 [5.88360656 2.74098361 4.38852459 1.43442623]
 [5.006      3.428      1.462      0.246     ]]
3
4


## Mutation algorithm 
this function do mutation algorithm on candidates interval's centers

In [91]:
def mutation(candidate_array,mutation_random):
    New_candidate = copy.deepcopy(candidate_array)
    Mutant_candidate = copy.deepcopy(candidate_array)
    Trial_candidate = copy.deepcopy(candidate_array)
    for x in range(len(candidate_array)):
        a = np.full(len(candidate_array), 1/(len(candidate_array)-1))
        a[x] = 0
        temp = np.random.choice(len(candidate_array), 3, replace = False, p=a)
        for i in range(len(candidate_array[0].centers)):
            for j in range(len(candidate_array[0].centers[i])):
                Mutant_candidate[x].centers[i][j] = candidate_array[temp[0]].centers[i][j]+ random.random()*(candidate_array[temp[1]].centers[i][j] - candidate_array[temp[2]].centers[i][j])
                if random.random() < mutation_random:
                    Trial_candidate[x].centers[i][j] = Mutant_candidate[x].centers[i][j] #Trial & Mutant must be copy of same candidate
                    
                    
        Trial_candidate[x].fitness = sum_of_squares(Trial_candidate[x].nodes,Trial_candidate[x].centers,Trial_candidate[x].labels)
        
        if Trial_candidate[x].fitness < sum_of_squares(candidate_array[x].nodes,candidates_array[2].centers,candidates_array[2].labels):
            New_candidate[x] = Trial_candidate[x]
            print("sakjfnr")
        else:
            New_candidate[x] = candidate_array[x]
        #print(Trial_candidate[x].centers)
        #print(candidate_array[x].centers)
        #print(New_candidate[x].centers)
        #print(Mutant_candidate[x].centers)
        print("###########################################################################################################")
        Trial_candidate[x].each_fitness = each_fitness(Trial_candidate[x])
        print(Trial_candidate[x].each_fitness)
        candidate_array[x].each_fitness = each_fitness(candidate_array[x])
        print(candidate_array[x].each_fitness)
        print("###########################################################################################################")
        
    return New_candidate
#print(candidates_array[1].fitness)
#for i in range(0,3500):
New_candidates_array = mutation(candidates_array,mutation_random)
    #if New_candidates_array[1].fitness != candidates_array[1].fitness:
     #   print(New_candidates_array[1].fitness)#never happen

###########################################################################################################
[[183.85863905  45.32946746  54.07285309 124.51563146]
 [114.92438167  49.15468625  57.23953362 104.32826228]
 [178.76223392  45.4946004   94.520528   168.320592  ]]
[[183.85863905  45.32946746 110.37976331  67.07786982]
 [117.25224133  48.07917227  58.83535071  92.93291051]
 [ 76.465232    45.350208    94.520528   168.320592  ]]
###########################################################################################################
###########################################################################################################
[[183.85863905  45.32946746 110.37976331  67.07786982]
 [117.25224133  48.07917227  58.83535071  92.93291051]
 [ 76.465232    45.350208    94.520528   168.320592  ]]
[[183.85863905  45.32946746 110.37976331  67.07786982]
 [117.25224133  48.07917227  58.83535071  92.93291051]
 [ 76.465232    45.350208    94.520528   168.320592  ]]
############

# Probability Equation

In [92]:
def probability(candidates):
    for c in range(0,len(candidates)):
        temp_array = []
        for clusters in range(0,len(candidates[c].centers)):
            temp_array.append((1/np.sum(candidates[c].each_fitness[clusters]))/(np.sum(1/(np.sum(x.each_fitness[clusters])) for x in candidates)))
        print(temp_array)    
        candidates[c].probability = temp_array
    return

### Roulette Wheel Selection
This is a logic for selecting targets with higher probability.<br>
In this article, each candidate want to follow the other candidate with higher fitness. So in the previous section <strong><em>Probability Equation<strong><em>, we calculated all probabilities of candidates based on their fitnesses.

In [93]:
def roulette_wheel_selection(inertia_array):
    maximum = np.sum(inertia_array)
    pick = random.uniform(0, maximum)
    current = 0
    for fitness in inertia_array:
        current += fitness
        if current > pick:
            return fitness

In [94]:
def shrinked_sampling_interval(candidates_array):
    for i in range(0,len(candidates_array)):
        shrinked_sampling_interval = []
        for j in range(0,len(candidates_array[i].centers)):
            Second_layer = []
            for k in range(0,len(candidates_array[i].centers[j])):
                Third_layer = []
                Temp = abs(candidates_array[i].sampling_interval[0][k] - candidates_array[i].sampling_interval[1][k]) * sampling_interval_reduction_factor
                Third_layer.append(candidates_array[i].centers[j][k] - (Temp/2))
                Third_layer.append(candidates_array[i].centers[j][k] + (Temp/2))
                Second_layer.append(Third_layer)
            shrinked_sampling_interval.append(Second_layer)
        candidates_array[i].sampling_interval = shrinked_sampling_interval
    return
shrinked_sampling_interval(New_candidates_array)
print(New_candidates_array[0].sampling_interval)

[[[5.143846153846154, 8.563846153846155], [1.936923076923077, 4.216923076923077], [2.9128846153846144, 8.517884615384615], [0.9138461538461538, 3.1938461538461533]], [[4.173606557377049, 7.593606557377049], [1.600983606557377, 3.8809836065573773], [1.5860245901639338, 7.191024590163934], [0.29442622950819697, 2.5744262295081968]], [[3.2960000000000003, 6.716], [2.288, 4.568], [-1.3405000000000005, 4.2645], [-0.8939999999999996, 1.3860000000000001]]]
